# Processing mixed beverage data
This Jupyter Notebook uses curl to down download [Mixed Beverage Gross Receipts](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php) files from the Texas Comptroller's [data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/), and then a python library called [agate](http://agate.readthedocs.io/) to clean and process that data for [stories similar to this one](http://www.mystatesman.com/business/austin-alcohol-sales-percent-february/Oo2txZUkuDlqBl0rU9O1lJ/) on monthly alcohol sales.

This is a stripped down version (compared to the original fork) that skips explanation of steps beyond commenting.

## Get to the goods

To skip most of the setup code and get to what you really want, search for:

- Top sales statewide
- Austin sales
- Central Texas cities

### File download

- Go to the [Texas Comptroller data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/) and copy the url for the CSV for this month and enter it below.
- You also need to set to set the [processing variables](Processing-variables) for this month.

In [1]:
%%bash
## downloads the mixedbev file into mixbev-files folder
## You have to set this URL based on location in data center

## these are commented out now to do manually when you want to run it
# cd ../test/
# curl -O https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_03_2017.CSV

In [1]:
# imports the libraries we will use
import agate
from decimal import Decimal
import re

# this surpresses the timezone warning
# Might comment out during development so other warnings
# are not surpressed
import warnings
warnings.filterwarnings('ignore')

### Processing variables
Then we set some values based on those.

- The **`file`** is the name of the file we want to process
- The **`tax_rate`** is the value we need for this file to get the Gross Recipts (vs the Tax Reported, which is just the tax amount the establishment paid). The comptroller [has information on the tax](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php), but this [old record layout](https://github.com/utdata/cli-tools/blob/master/data/mixbevtax/OLD-MIXEDBEVTAX-LAYOUT.txt) best describes the math.
- The **`month_studied`** is the YYYY/MM designation for the month before the file release. The file released in February has mostly records from January, but can also have any other month, so we set here the specific month we want. Note there is a check later on that counts the number of files by month, which is worth checking.

Here are the files we have downloaded:

In [2]:
ls ../mixbev-files/2017/

MIXEDBEV_02_2017.CSV  MIXEDBEV_04_2015.CSV  MIXEDBEV_05_2017.CSV
MIXEDBEV_03_2017.CSV  MIXEDBEV_04_2017.CSV


In [3]:
# this is our source file, which may have been downloaded above
# Swap out the file name here and date below as needed
file = '../mixbev-files/2017/MIXEDBEV_05_2017.CSV'
file_name = ''
online_name = ''

# setting the month_studied var.
# This should be checked in the table below that counts records by month
month_studied = '2017/04'

# Sets the tax rate to convert Report Tax to Gross Receipts
# It's 6.7 since January 1, 2014
tax_rate = Decimal('6.7')

### import and processing

In [4]:
# sets the column names of the original data set.
column_names = [
    'TABC Permit Number',
    'Trade Name',
    'Location Address',
    'Location City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Blank',
    'Report Period',
    'Report Tax'
]
# Helps us import some text fields that may be considered numbers in error.
specified_types = {
    'Location Zip Code': agate.Text(),
    'Location County Code': agate.Text()
}

# this imports the file specified above, along with the proper types
mixbev_raw = agate.Table.from_csv(file, column_names, encoding='iso-8859-1', column_types=specified_types)

# mixbev_trim creates a new interim table with results of compute function
# that takes the four columns that need trimming and strips them of white space,
# adding them to the end of the table with new names.
# The last computation does the math to create the Gross Receipts based on the tax_rate set above

mixbev_trim = mixbev_raw.compute([
    ('Permit', agate.Formula(agate.Text(), lambda r: r['TABC Permit Number'].strip())),
    ('Name', agate.Formula(agate.Text(), lambda r: r['Trade Name'].strip())),
    ('Address', agate.Formula(agate.Text(), lambda r: r['Location Address'].strip())),
    ('City', agate.Formula(agate.Text(), lambda r: r['Location City'].strip())),
    ('Receipts_compute', agate.Formula(agate.Number(), lambda r: (r['Report Tax'] / tax_rate) * 100))
])

# the Receipts_compute computation above returns as a decimal number,
# so this function rounds those numbers.
# I might refactor this late so I can use it elsewhere.
def round_receipt(row):
    return row['Receipts_compute'].quantize(Decimal('0.01'))

# This compute method uses round_recipt function above,
# putting the results into a new table.
mixbev_round = mixbev_trim.compute([
    ('Receipts', agate.Formula(agate.Number(), round_receipt))
])

# creates new table, selecting just the columns we need
# then renames some of them for ease later.
mixbev_cleaned = mixbev_round.select([
    'Permit',
    'Name',
    'Address',
    'City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Report Period',
    'Report Tax',
    'Receipts'
]).rename(column_names = {
    'Location State': 'State',
    'Location Zip Code': 'Zip',
    'Location County Code': 'CountyCode',
    'Report Period': 'Period',
    'Report Tax': 'Tax'
})

# Concatenates the name and address
mixbev_cleaned_est = mixbev_cleaned.compute([
    ('Establishment', agate.Formula(agate.Text(), lambda row: '%(Name)s %(Address)s' % row))
])

# importing countes.csv, ensuring that the 'code' column is text
counties = agate.Table.from_csv('../resource-files/counties.csv', column_types={'code': agate.Text()})

# joines the counties table to the mixed bev cleaned data with establishments
mixbev_joined = mixbev_cleaned_est.join(counties, 'CountyCode', 'code')

# get just the columns we need and rename county
# THIS is the finished, cleaned mixbev table
mixbev = mixbev_joined.select([
    'Permit',
    'Name',
    'Address',
    'Establishment',
    'City',
    'State',
    'Zip',
    'county',
    'Period',
    'Tax',
    'Receipts'
]).rename(column_names = {
    'county': 'County'
})


### Looking at dates of the records

This basically confirms that the file has multiple dates, and that we are looking at the right month of data. Typically a data set will have mostly reports from the previous month, but there are always also submissions from other months. We want to filter out those other months, which we do based on the `month_studied` variable set near the top of the file, which should match the period at the top of the table below.


In [5]:
# Pivot the mixbev table by Period. Default it give a Count of the records
# We then order the table by Count in descending order
by_period = mixbev.pivot('Period').order_by('Count', reverse=True)

# prints the table of period and number of records
by_period.limit(5).print_table(max_rows=None)

| Period  |  Count |
| ------- | ------ |
| 2017/04 | 14,410 |
| 2017/03 |  1,819 |
| 2017/02 |    187 |
| 2017/01 |     52 |
| 2017/05 |     35 |


In [6]:
## filters the records to our month_studied
mixbev_month = mixbev.where(lambda row: row['Period'] == month_studied)

# function to group sales by a specific location
# City or County passed in should be ALL CAPS
# Location_type can be 'City' or 'County'

def location_sum(location_type, location):
    # Filters the data to the specified city
    location_filtered = mixbev_month.where(lambda row: row[location_type].upper() == location)

    # groups the data based on Establishment and location
    location_grouped = location_filtered.group_by('Establishment').group_by(location_type)
    # computes the sales based on the grouping
    location_summary = location_grouped.aggregate([
        ('Tax_sum', agate.Sum('Tax')),
        ('Receipts_sum', agate.Sum('Receipts'))
    ])
    
    # sorts the results by most sold
    location_summary_sorted = location_summary.order_by('Receipts_sum', reverse=True)
    # prints the top 10 results
    
    return(location_summary_sorted)


## Top sales statewide

Because we want to group our results by more than one field and perform more than one aggregation, we'll do this a little differently. We'll use group_by to create a grouped table, then perform aggregations on that new table to computer the Tax and Receipts columns.

In [7]:
# groups the data based on Establishment and City
mixbev_grouped = mixbev_month.group_by('Establishment').group_by('County').group_by('City')

# computes the sales based on the grouping
state_summary = mixbev_grouped.aggregate([
    ('Tax_sum', agate.Sum('Tax')),
    ('Sales_sum', agate.Sum('Receipts'))
])

# sorts the results by most sold. We could probalby chain it above if we wanted to.
state_summary_sorted = state_summary.order_by('Sales_sum', reverse=True)

# summing sales statewide for month
print('\nTotal sales across the state for the given month: {}\n'.format(
    mixbev_month.aggregate(agate.Sum('Receipts'))
))

print('Top sales by establishment statewide\n')

# prints the top 10 results
state_summary_sorted.limit(10).print_table(max_column_width=40)


Total sales across the state for the given month: 559110269.76

Top sales by establishment statewide

| Establishment                            | County     | City            |    Tax_sum |    Sales_sum |
| ---------------------------------------- | ---------- | --------------- | ---------- | ------------ |
| HOSPITALITY INTERNATIONAL, INC 23808 ... | Bexar      | SAN ANTONIO     | 115,379.22 | 1,722,077.91 |
| GAYLORD TEXAN 1501 GAYLORD TRL           | Tarrant    | GRAPEVINE       | 110,838.77 | 1,654,310.00 |
| SPECTRUM CATERING 27328 SPECTRUM WAY     | Montgomery | OAK RIDGE NORTH |  94,192.62 | 1,405,860.00 |
| WLS BEVERAGE CO 110 E 2ND ST             | Travis     | AUSTIN          |  72,186.73 | 1,077,413.88 |
| METROPLEX SPORTSERVICE, INC. 1000 BAL... | Tarrant    | ARLINGTON       |  71,724.84 | 1,070,520.00 |
| LEVY RESTAURANTS AT TOYOTA CEN 1510 P... | Harris     | HOUSTON         |  70,604.80 | 1,053,802.99 |
| OMNI DALLAS CONVENTION CENTER 555 S L... | Dallas     | DALLAS 

## Austin sales and sums

With this, we refernce the location_sum function above, and pass the type of location (City) and the name of the city (AUSTIN). At the same time, we limit the result of that function to the first 10 records, and then print the results. We are basically stringing together a bunch of stuff at once.

In [9]:
# uses the city_sum function to filter
austin = location_sum('City', 'AUSTIN')

print('\nTotal sales across the state for the given month: {}\n'.format(
    austin.aggregate(agate.Sum('Receipts_sum'))
))

# print the resulting table
austin.limit(5).print_table(max_column_width=40)


Total sales across the state for the given month: 66175937.16

| Establishment                            | City   |   Tax_sum | Receipts_sum |
| ---------------------------------------- | ------ | --------- | ------------ |
| WLS BEVERAGE CO 110 E 2ND ST             | AUSTIN | 72,186.73 | 1,077,413.88 |
| RYAN SANDERS SPORTS SERVICES, 9201 CI... | AUSTIN | 58,267.08 |   869,657.91 |
| ROSE ROOM/ 77 DEGREES 11500 ROCK ROSE... | AUSTIN | 34,129.59 |   509,396.87 |
| 400 BAR/CUCARACHA/CHUPACABRA/J 400 E ... | AUSTIN | 31,458.10 |   469,523.88 |
| FARACE BEVERAGES, LLC 310 WILLIE NELS... | AUSTIN | 30,531.09 |   455,687.91 |


## More Central Texas cities

In [10]:
location_sum('City', 'BASTROP').limit(3).print_table(max_column_width=40)

| Establishment                            | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------- | -------- | ------------ |
| OLD TOWN RESTURANT AND BAR/PIN 931 MA... | BASTROP | 4,473.25 |    66,764.93 |
| CHILI'S GRILL & BAR 734 HIGHWAY 71 W     | BASTROP | 3,691.03 |    55,090.00 |
| NEIGHBOR'S 601 CHESTNUT ST UNIT C        | BASTROP | 2,556.58 |    38,157.91 |


In [11]:
location_sum('City', 'BEE CAVE').limit(3).print_table(max_column_width=40)

| Establishment                            | City     |  Tax_sum | Receipts_sum |
| ---------------------------------------- | -------- | -------- | ------------ |
| HCG BEVERAGE, LLC 12525 BEE CAVE PKWY    | BEE CAVE | 6,951.78 |   103,757.91 |
| WOODY TAVERN AND GRILL, INC. 12801 SH... | BEE CAVE | 6,506.90 |    97,117.91 |
| CAFE BLUE 12800 HILL COUNTRY BLVD STE    | BEE CAVE | 6,160.38 |    91,945.97 |


In [12]:
location_sum('City', 'BUDA').limit(3).print_table(max_column_width=40)

| Establishment                  | City |  Tax_sum | Receipts_sum |
| ------------------------------ | ---- | -------- | ------------ |
| BUCKS BACKYARD 1750 S FM 1626  | BUDA | 8,329.64 |   124,322.99 |
| WILLIE'S JOINT 824 MAIN ST     | BUDA | 4,347.09 |    64,881.94 |
| PINBALLZ KINGDOM 15201 S IH 35 | BUDA | 3,481.18 |    51,957.91 |


In [13]:
location_sum('City', 'CEDAR PARK').limit(3).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| CHUY'S 4911 183A TOLL RD                 | CEDAR PARK | 9,008.68 |   134,457.91 |
| LUPE TORTILLA MEXICAN RESTAURA 4501 1... | CEDAR PARK | 7,555.72 |   112,771.94 |
| BJ'S RESTAURANT & BREWERY 1001 E WHIT... | CEDAR PARK | 5,551.41 |    82,856.87 |


In [14]:
location_sum('City', 'DRIPPING SPRINGS').limit(3).print_table(max_column_width=40)

| Establishment                            | City             |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------------- | -------- | ------------ |
| TRUDY'S FOUR STAR 13059 FOUR STAR BLVD   | DRIPPING SPRINGS | 5,168.24 |    77,137.91 |
| DEEP EDDY DISTILLING CO 2250 E HIGHWA... | DRIPPING SPRINGS | 4,862.59 |    72,575.97 |
| FLORES MEXICAN RESTAURANT 2440 E HIGH... | DRIPPING SPRINGS | 3,998.42 |    59,677.91 |


In [15]:
location_sum('City', 'GEORGETOWN').limit(3).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| EL MONUMENTO 205 W 2ND ST                | GEORGETOWN | 6,767.06 |   101,000.90 |
| GATEWAY BEVERAGES, INC 1101 WOODLAWN AVE | GEORGETOWN | 5,579.82 |    83,280.90 |
| HARDTAILS 1515 N IH 35                   | GEORGETOWN | 4,867.88 |    72,654.93 |


In [16]:
location_sum('City', 'KYLE').limit(3).print_table(max_column_width=40)

| Establishment                            | City |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---- | -------- | ------------ |
| CASA GARCIA'S MEXICAN RESTAURA 5401 F... | KYLE | 5,188.61 |    77,441.94 |
| EVO ENTERTAINMENT CENTER 3200 KYLE XING  | KYLE | 3,161.52 |    47,186.87 |
| CENTERFIELD SPORTS BAR & GRILL 200 W ... | KYLE | 2,604.69 |    38,875.97 |


In [17]:
location_sum('City', 'LAGO VISTA').limit(3).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| COPPERHEAD GRILL 6115 LOHMANS FORD RD    | LAGO VISTA | 1,293.56 |    19,306.87 |
| MARIA'S BAR & GRILL MEXICAN RE 20602 ... | LAGO VISTA |   961.38 |    14,348.96 |


In [18]:
location_sum('City', 'LAKEWAY').limit(3).print_table(max_column_width=40)

| Establishment                            | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------- | -------- | ------------ |
| LAKEWAY RESORT AND SPA 101 LAKEWAY DR    | LAKEWAY | 8,525.95 |   127,252.99 |
| THE GROVE WINE BAR AND KITCHEN 3001 R... | LAKEWAY | 8,203.07 |   122,433.88 |
| CANYON GRILLE AT ROUGH HOLLOW, 103 YA... | LAKEWAY | 4,131.68 |    61,666.87 |


In [19]:
location_sum('City', 'LEANDER').limit(3).print_table(max_column_width=40)

| Establishment                            | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------- | -------- | ------------ |
| BROOKLYN HEIGHTS PIZZERIA 3550 LAKELI... | LEANDER | 3,768.34 |    56,243.88 |
| JARDIN DEL REY 703 S HIGHWAY 183         | LEANDER | 2,802.14 |    41,822.99 |
| TAPATIA JALISCO #3 LLC 651 N US 183      | LEANDER | 1,080.44 |    16,125.97 |


In [20]:
location_sum('City', 'LIBERTY HILL').limit(3).print_table(max_column_width=40)

| Establishment                            | City         |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------------ | -------- | ------------ |
| JARDIN CORONA 15395 W STATE HIGHWAY 29   | LIBERTY HILL | 2,979.28 |    44,466.87 |
| MARGARITA'S RESTAURANT 10280 W STATE ... | LIBERTY HILL | 2,414.47 |    36,036.87 |
| ELENAS MEXICAN RESTAURANT 14801 W STA... | LIBERTY HILL |   216.94 |     3,237.91 |


In [21]:
location_sum('City', 'PFLUGERVILLE').limit(3).print_table(max_column_width=40)

| Establishment                          | City         |  Tax_sum | Receipts_sum |
| -------------------------------------- | ------------ | -------- | ------------ |
| MAVERICKS 1700 GRAND AVENUE PKWY STE 2 | PFLUGERVILLE | 8,799.98 |   131,342.99 |
| HANOVER'S DRAUGHT HAUS 108 E MAIN ST   | PFLUGERVILLE | 4,841.82 |    72,265.97 |
| LAST CALL 1615 GRAND AVENUE PKWY STE 2 | PFLUGERVILLE | 4,373.96 |    65,282.99 |


In [22]:
location_sum('City', 'ROUND ROCK').limit(5).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| THIRD BASE ROUND ROCK, LLC 3107 S INT... | ROUND ROCK | 9,124.12 |   136,180.90 |
| JACK ALLEN'S KITCHEN 2500 HOPPE TRL      | ROUND ROCK | 8,959.03 |   133,716.87 |
| CHUY'S ROUND ROCK 2320 N INTERSTATE 35   | ROUND ROCK | 8,787.98 |   131,163.88 |
| TWIN PEAKS RESTAURANT 100 LOUIS HENNA... | ROUND ROCK | 8,557.10 |   127,717.91 |
| RICK'S CABARET 3105 S INTERSTATE 35      | ROUND ROCK | 8,554.76 |   127,682.99 |


In [23]:
location_sum('City', 'SAN MARCOS').limit(5).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| ZELICKS 336 W HOPKINS ST                 | SAN MARCOS | 9,251.09 |   138,075.97 |
| PLUCKERS WING BAR 105 N INTERSTATE 35    | SAN MARCOS | 7,929.98 |   118,357.91 |
| THE MARC 120 E SAN ANTONIO ST            | SAN MARCOS | 7,553.78 |   112,742.99 |
| CHIMY'S SAN MARCOS 217 E HOPKINS ST      | SAN MARCOS | 7,227.62 |   107,874.93 |
| THE TAP ROOM & THE PORCH ON HO 129 E ... | SAN MARCOS | 6,395.88 |    95,460.90 |


In [24]:
location_sum('City', 'SPICEWOOD').limit(3).print_table(max_column_width=40)

| Establishment                            | City      |  Tax_sum | Receipts_sum |
| ---------------------------------------- | --------- | -------- | ------------ |
| ANGEL'S ICEHOUSE 21815 W HWY 71          | SPICEWOOD | 4,329.87 |    64,624.93 |
| POODIES HILLTOP ROADHOUSE 22308 STATE... | SPICEWOOD | 3,419.74 |    51,040.90 |
| APIS RESTAURANT 23526 STATE HIGHWAY 71 W | SPICEWOOD | 1,985.00 |    29,626.87 |


In [25]:
location_sum('City', 'SUNSET VALLEY').limit(3).print_table(max_column_width=40)

| Establishment                            | City          |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------------- | -------- | ------------ |
| DOC'S BACKYARD 5207 BRODIE LN # 100      | SUNSET VALLEY | 5,567.56 |    83,097.91 |
| BJ'S RESTAURANT AND BREWHOUSE 5207 BR... | SUNSET VALLEY | 4,401.96 |    65,700.90 |
| CANE ROSSO 4715 S LAMAR BLVD STE 103     | SUNSET VALLEY | 1,953.98 |    29,163.88 |


In [26]:
location_sum('City', 'WEST LAKE HILLS').limit(3).print_table(max_column_width=40)

| Establishment                            | City            |  Tax_sum | Receipts_sum |
| ---------------------------------------- | --------------- | -------- | ------------ |
| LUPE TORTILLA MEXICAN RESTAURA 701 S ... | WEST LAKE HILLS | 6,330.62 |    94,486.87 |
| CHIPOTLE CHIPOTLE MEXICAN GRIL 3300 B... | WEST LAKE HILLS |    43.34 |       646.87 |


## Sales by county example

In this case, we pass in the location type of 'County' and then a county name in caps to get the most sales in a particular county.

In [27]:
location_sum('County', 'CALDWELL').limit(3).print_table(max_column_width=40)

| Establishment                            | County   |  Tax_sum | Receipts_sum |
| ---------------------------------------- | -------- | -------- | ------------ |
| GUADALAJARA MEXICAN RESTAURANT 1710 S... | Caldwell | 1,336.18 |    19,942.99 |
| RISKY BUSINESS 211 E MARKET ST           | Caldwell | 1,326.53 |    19,798.96 |
| THE PEARL 110 N MAIN ST                  | Caldwell |   633.48 |     9,454.93 |


## Sales by ZIP Code
Just making sure that 78701 is at the top of this list, which it has been every month for a decade.

In [28]:
# top zip code gross receipts
zip_receipts = mixbev_month.pivot('Zip', aggregation=agate.Sum('Receipts')).order_by('Sum', reverse=True)
zip_receipts.limit(5).print_table()

| Zip   |           Sum |
| ----- | ------------- |
| 78701 | 25,024,956.58 |
| 75201 | 11,936,866.46 |
| 78205 | 11,661,569.95 |
| 77002 |  9,664,799.57 |
| 78704 |  7,733,703.75 |
